import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.functions import desc
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [2]:
spark = SparkSession.builder.appName("LocalSparkSession").master("local[*]").getOrCreate()

print("pySpark Version:", spark.version)



pySpark Version: 3.5.3


read csv with inferschema

In [3]:
df = spark.read.options(inferSchema='True').csv("ds_salaries.csv")

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [4]:
df = spark.read.csv("ds_salaries.csv")

write schema of scv on screen

In [5]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



create schema of this scv

In [6]:
schema = StructType([
    StructField("work_year", IntegerType(), True),  
    StructField("experience_level", StringType(), True),  
    StructField("employment_type", StringType(), True),  
    StructField("job_title", StringType(), True),  
    StructField("salary", IntegerType(), True),  
    StructField("salary_currency", StringType(), True),  
    StructField("salary_in_usd", IntegerType(), True),  
    StructField("employee_residence", StringType(), True),  
    StructField("remote_ratio", IntegerType(), True),  
    StructField("company_location", StringType(), True),  
    StructField("company_size", StringType(), True)  
])


df = spark.read.options(header=True, inferSchema=False,schema =schema).csv("ds_salaries.csv")


df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:
df_infer_schema = spark.read.options(header=True, inferSchema=True).csv("ds_salaries.csv")
df_infer_schema.printSchema()


root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [8]:
df_infer_schema.printSchema()


root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [9]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [10]:
display(df.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


print all rows from df_job_titles without truncating jobs

In [11]:
df.select("job_title").distinct().show(truncate=False)


+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [12]:
df_analytic = df.groupBy("job_title").agg(
    avg("salary_in_usd").alias("avg_salary"),
    min("salary_in_usd").alias("min_salary"),
    max("salary_in_usd").alias("max_salary")
)

df_analytic.show(truncate=False)

+----------------------------------+------------------+----------+----------+
|job_title                         |avg_salary        |min_salary|max_salary|
+----------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher     |5409.0            |5409      |5409      |
|AI Scientist                      |66135.57142857143 |12000     |55000     |
|Analytics Engineer                |175000.0          |135000    |205300    |
|Applied Data Scientist            |175655.0          |110037    |54238     |
|Applied Machine Learning Scientist|142068.75         |31875     |75000     |
|BI Data Analyst                   |74755.16666666667 |100000    |98000     |
|Big Data Architect                |99703.0           |99703     |99703     |
|Big Data Engineer                 |51974.0           |109024    |70000     |
|Business Data Analyst             |76691.2           |100000    |70912     |
|Cloud Data Engineer               |124647.0          |160000   

print all rows from df_analytic without trancating jobs

In [13]:
df_analytic.show(n=df_analytic.count(), truncate=False)



+----------------------------------------+------------------+----------+----------+
|job_title                               |avg_salary        |min_salary|max_salary|
+----------------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher           |5409.0            |5409      |5409      |
|AI Scientist                            |66135.57142857143 |12000     |55000     |
|Analytics Engineer                      |175000.0          |135000    |205300    |
|Applied Data Scientist                  |175655.0          |110037    |54238     |
|Applied Machine Learning Scientist      |142068.75         |31875     |75000     |
|BI Data Analyst                         |74755.16666666667 |100000    |98000     |
|Big Data Architect                      |99703.0           |99703     |99703     |
|Big Data Engineer                       |51974.0           |109024    |70000     |
|Business Data Analyst                   |76691.2           |100000    |7091

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [14]:
window_spec = Window.orderBy(desc("avg_salary"))

df_analytic = df_analytic.withColumn("row_id", row_number().over(window_spec))

df_analytic.show(truncate=False)

+----------------------------------+------------------+----------+----------+------+
|job_title                         |avg_salary        |min_salary|max_salary|row_id|
+----------------------------------+------------------+----------+----------+------+
|Data Analytics Lead               |405000.0          |405000    |405000    |1     |
|Principal Data Engineer           |328333.3333333333 |185000    |600000    |2     |
|Financial Data Analyst            |275000.0          |100000    |450000    |3     |
|Principal Data Scientist          |215242.42857142858|148261    |416000    |4     |
|Director of Data Science          |195074.0          |130026    |325000    |5     |
|Data Architect                    |177873.9090909091 |144854    |90700     |6     |
|Applied Data Scientist            |175655.0          |110037    |54238     |7     |
|Analytics Engineer                |175000.0          |135000    |205300    |8     |
|Data Specialist                   |165000.0          |165000    

print all data from df_analytic

In [15]:
df_analytic.show(n=df_analytic.count(), truncate=False)

+----------------------------------------+------------------+----------+----------+------+
|job_title                               |avg_salary        |min_salary|max_salary|row_id|
+----------------------------------------+------------------+----------+----------+------+
|Data Analytics Lead                     |405000.0          |405000    |405000    |1     |
|Principal Data Engineer                 |328333.3333333333 |185000    |600000    |2     |
|Financial Data Analyst                  |275000.0          |100000    |450000    |3     |
|Principal Data Scientist                |215242.42857142858|148261    |416000    |4     |
|Director of Data Science                |195074.0          |130026    |325000    |5     |
|Data Architect                          |177873.9090909091 |144854    |90700     |6     |
|Applied Data Scientist                  |175655.0          |110037    |54238     |7     |
|Analytics Engineer                      |175000.0          |135000    |205300    |8     |

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [16]:
df_analytic = df_analytic.select("row_id", "job_title", "avg_salary", "min_salary", "max_salary")

print df_analytic now

In [17]:
df_analytic.show(truncate=False)

+------+----------------------------------+------------------+----------+----------+
|row_id|job_title                         |avg_salary        |min_salary|max_salary|
+------+----------------------------------+------------------+----------+----------+
|1     |Data Analytics Lead               |405000.0          |405000    |405000    |
|2     |Principal Data Engineer           |328333.3333333333 |185000    |600000    |
|3     |Financial Data Analyst            |275000.0          |100000    |450000    |
|4     |Principal Data Scientist          |215242.42857142858|148261    |416000    |
|5     |Director of Data Science          |195074.0          |130026    |325000    |
|6     |Data Architect                    |177873.9090909091 |144854    |90700     |
|7     |Applied Data Scientist            |175655.0          |110037    |54238     |
|8     |Analytics Engineer                |175000.0          |135000    |205300    |
|9     |Data Specialist                   |165000.0          |165

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [18]:
max_salaries = df.groupBy("experience_level").agg(
    max("salary_in_usd").alias("biggest_salary")
)
df_exp_lvl = df.join(max_salaries, 
                     (df.experience_level == max_salaries.experience_level) & 
                     (df.salary_in_usd == max_salaries.biggest_salary), 
                     "inner")




print here df_exp_lvl

In [19]:
df_exp_lvl.show(truncate=False)

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+
|id |work_year|experience_level|employment_type|job_title           |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|experience_level|biggest_salary|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+
|161|2021     |EX              |FT             |Head of Data Science|85000 |USD            |85000        |RU                |0           |RU              |M           |EX              |85000         |
|196|2021     |EN              |FT             |BI Data Analyst     |9272  |USD            |9272         |KE                |100         |KE              |S           |EN              |9272       

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [20]:

max_salaries = df.groupBy("experience_level").agg(
    max("salary_in_usd").alias("biggest_salary")
)
max_salaries = max_salaries.withColumnRenamed("experience_level", "exp_level_max")

df_best = df.join(
    max_salaries,
    (df.experience_level == max_salaries.exp_level_max) &
    (df.salary_in_usd == max_salaries.biggest_salary),
    "inner"
).select(
    col("id"),
    col("biggest_salary"),
    col("experience_level"),
    col("employee_residence")
)
df_best.show(truncate=False)



+---+--------------+----------------+------------------+
|id |biggest_salary|experience_level|employee_residence|
+---+--------------+----------------+------------------+
|161|85000         |EX              |RU                |
|196|9272          |EN              |KE                |
|255|99703         |SE              |CA                |
|543|99100         |MI              |US                |
+---+--------------+----------------+------------------+



drop duplicates if exist by experience_level

In [21]:
df_best = df_best.distinct()

print df_best

In [22]:
df_best.show(truncate=False)

+---+--------------+----------------+------------------+
|id |biggest_salary|experience_level|employee_residence|
+---+--------------+----------------+------------------+
|196|9272          |EN              |KE                |
|543|99100         |MI              |US                |
|255|99703         |SE              |CA                |
|161|85000         |EX              |RU                |
+---+--------------+----------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [23]:
df_new_best = df_best.withColumn("experience_level", when(df_best.experience_level == "MI","middle")
                                 .when(df_best.experience_level == "SE","senior")).select(col("biggest_salary"),col("experience_level"),col("employee_residence"))


print df_new_best

In [24]:
df_new_best.show(truncate=False)

+--------------+----------------+------------------+
|biggest_salary|experience_level|employee_residence|
+--------------+----------------+------------------+
|9272          |NULL            |KE                |
|99100         |middle          |US                |
|99703         |senior          |CA                |
|85000         |NULL            |RU                |
+--------------+----------------+------------------+



write df_new_best like 1.csv and load then it to df_final

In [25]:
df_pandas = df_new_best.toPandas()
df_pandas.to_csv("1.csv", index=False)

print df_final

In [45]:
df_final = spark.read.options(header=True).csv("1.csv")
df_final.show()

+--------------+----------------+------------------+
|biggest_salary|experience_level|employee_residence|
+--------------+----------------+------------------+
|          9272|            NULL|                KE|
|         99100|          middle|                US|
|         99703|          senior|                CA|
|         85000|            NULL|                RU|
+--------------+----------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [46]:
df_filtered = df_final.filter(df_final.experience_level.isNotNull())
df_filtered = df_filtered.withColumnRenamed("biggest_salary","sal")
df_final = df_filtered.join(
    df,
    (df_filtered.sal == df.salary_in_usd) & 
    (df_filtered.employee_residence == df.employee_residence),
    how="inner"
)



print df_final

In [47]:
df_final.show()

+-----+----------------+------------------+---+---------+----------------+---------------+------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|  sal|experience_level|employee_residence| id|work_year|experience_level|employment_type|         job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+-----+----------------+------------------+---+---------+----------------+---------------+------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|99703|          senior|                CA|255|     2021|              SE|             FT|Big Data Architect|125000|            CAD|        99703|                CA|          50|              CA|           M|
|99100|          middle|                US|543|     2022|              MI|             FT|     Data Engineer| 99100|            USD|        99100|                US

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [49]:
max_salary_row = df_final.agg(max("salary_in_usd").alias("biggest_salary")).collect()[0]
biggest_salary = max_salary_row["biggest_salary"]
print("The biggest salary in USD from df_final is:", biggest_salary)


The biggest salary in USD from df_final is: 99703


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system